# Detección de movimiento en un video

Existen múltiples maneras de detectar el movimiento, la más básica es suponer que en el primer cuadro del video no hay movimiento y a partir de ahí, revisar qué tan diferente son los demás frames respecto al primero.
Si son muy diferentes (especificado por algún threshold), entonces se toma como que hubo algún movimiento.  
La desventaja con este método es que incluso en un lugar donde no hay movimiento, pueden haber cambios, por ejemplo, las condiciones de luz pueden variar conforme avanza el día.
Para mejorar este método lo que se hace es tomar el fondo del video (este se determina a partir de los últimos n frames del video que se han visto) y ver hay algún objeto que haya cambiado.

En el presente notebook se muestra una forma de realizar este último procedimiento con el uso de OpenCV y uno de los substractores de fondo que ya vienen implementados en la biblioteca.

In [1]:
import imutils
import cv2

In [2]:
# Sirve para dibujar un rectángulo alrededor de cada contorno detectado
def draw_rectangles(contours, frame, min_area = 500, color=(0, 255, 0), thickness=2):
    occupied = False
    for contour in contours:
        if cv2.contourArea(contour) < min_area:
            continue
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, thickness)
        occupied  = True
    return occupied

In [5]:
vid = cv2.VideoCapture(0)
min_area = 3000
text_font = cv2.FONT_HERSHEY_SIMPLEX

# Este objeto nos va a ayudar a extraer el fondo de cada cuadro del video
fgbg = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = vid.read()
    if not ret:
        break

    # Normalizamos los frames a 500px de ancho
    frame = imutils.resize(frame, width=500)

    # Los pasamos a escala de grises y hacemos un suavizado
    grayscale_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    grayscale_frame = cv2.GaussianBlur(grayscale_frame, (21, 21), 0)

    # Extraemos el fondo, y nos queda sólo aquellos objetos que están al frente y podrían estar en movimiento
    fgmask = fgbg.apply(frame)

    # Sólo aquellas partes del frente pasen el corte se consideran en movimiento
    thresh = cv2.threshold(fgmask, 25, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)

    contours = cv2.findContours(
        thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    occupied = draw_rectangles(contours, frame, min_area)

    cv2.imshow("Security Feed", frame)
    cv2.imshow("Threshold", thresh)
    cv2.imshow("Mask", fgmask)

    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break
vid.release()
cv2.destroyAllWindows()

## Fuentes

- [Visual tracking of human visitors under variable-lighting conditions for a responsive audio art installation](https://ieeexplore.ieee.org/document/6315174)
- [How to Use Background Subtraction Methods](https://docs.opencv.org/4.x/d1/dc5/tutorial_background_subtraction.html)
- [Basic motion detection and tracking with Python and OpenCV](https://pyimagesearch.com/2015/05/25/basic-motion-detection-and-tracking-with-python-and-opencv/)